In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import datetime as dt
from random import seed
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
tf.compat.v1.experimental.output_all_intermediates(True)

In [2]:
tf.random.set_seed(36)

## Dataset

In [3]:
BASE_DIR_PATH = '/Users/neilb/Documents/dsci_thesis/Datasets'
DATASET_FILE = os.path.join(BASE_DIR_PATH, 'compiled_data_2016_2017.csv')

In [4]:
df = pd.read_csv(DATASET_FILE)
df

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420


In [5]:
df = df[['Waterlevel_Sto_Nino']]
df.head()

,Waterlevel_Sto_Nino
0,12.18
1,12.19
2,12.19
3,12.20
4,12.20


In [6]:
# Splitting for time series: split into 50-25-25
n = len(df)
train_df = df[0:int(n*0.5)]
val_df = df[int(n*0.5):int(n*0.75)]
test_df = df[int(n*0.75):]

## Window Generator

In [7]:
BATCH_SIZE = 32

In [8]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None, batch_size=BATCH_SIZE):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
        self.batch_size = batch_size

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    # Creating tf datasets for more convenient use and integration into model in the future
    def make_dataset(self, data, shuffle=True):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=shuffle,
            batch_size=self.batch_size,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df, False)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [9]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        label_columns=['Waterlevel_Sto_Nino']
    )

wide_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [1 2 3 4 5 6]
Label column name(s): ['Waterlevel_Sto_Nino']

In [10]:
for example_inputs, example_labels in wide_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 1)
Labels shape (batch, time, features): (32, 6, 1)


In [11]:
wide_window.example

(<tf.Tensor: shape=(32, 6, 1), dtype=float32, numpy=
 array([[[12.58],
         [12.58],
         [12.58],
         [12.58],
         [12.58],
         [12.58]],
 
        [[12.15],
         [12.14],
         [12.14],
         [12.13],
         [12.13],
         [12.12]],
 
        [[11.91],
         [11.91],
         [11.91],
         [11.91],
         [11.91],
         [11.91]],
 
        [[12.49],
         [12.49],
         [12.49],
         [12.49],
         [12.49],
         [12.49]],
 
        [[13.3 ],
         [13.24],
         [13.18],
         [13.12],
         [13.08],
         [13.04]],
 
        [[12.6 ],
         [12.6 ],
         [12.6 ],
         [12.6 ],
         [12.6 ],
         [12.6 ]],
 
        [[11.31],
         [11.31],
         [11.31],
         [11.31],
         [11.31],
         [11.31]],
 
        [[11.89],
         [11.89],
         [11.89],
         [11.89],
         [11.89],
         [11.89]],
 
        [[13.01],
         [12.9 ],
         [12.8 ],
     

In [12]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['Waterlevel_Sto_Nino']
    )

conv_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): ['Waterlevel_Sto_Nino']

## Metrics

In [13]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [14]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Training Loop

In [15]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

## Models

In [16]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

## Training

In [17]:
dense_history = compile_and_fit(dense, wide_window)

Epoch 1/20
274/274 [==============================] - 5s 9ms/step - loss: 9.9879 - mean_squared_error: 9.9879 - NSE: -28.3122 - r_square: 0.9906 - val_loss: 0.0023 - val_mean_squared_error: 0.0023 - val_NSE: 0.9679 - val_r_square: 0.9742
Epoch 2/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - NSE: 0.9895 - r_square: 0.9909 - val_loss: 0.0022 - val_mean_squared_error: 0.0022 - val_NSE: 0.9672 - val_r_square: 0.9731
Epoch 3/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - NSE: 0.9895 - r_square: 0.9909 - val_loss: 0.0023 - val_mean_squared_error: 0.0023 - val_NSE: 0.9681 - val_r_square: 0.9744
Epoch 4/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - NSE: 0.9892 - r_square: 0.9907 - val_loss: 0.0024 - val_mean_squared_error: 0.0024 - val_NSE: 0.9653 - val_r_square: 0.9744


In [18]:
dense_history.history

{'loss': [9.987945556640625,
  0.004476659931242466,
  0.004484320525079966,
  0.004490440711379051],
 'mean_squared_error': [9.987937927246094,
  0.004476659931242466,
  0.0044843233190476894,
  0.004490438848733902],
 'NSE': [-28.31217384338379,
  0.9895405769348145,
  0.9894545078277588,
  0.9892172813415527],
 'r_square': [0.9906029105186462,
  0.9909492135047913,
  0.9908682107925415,
  0.9907158613204956],
 'val_loss': [0.0022752927616238594,
  0.0021952714305371046,
  0.0022625382989645004,
  0.0024402893614023924],
 'val_mean_squared_error': [0.0022752925287932158,
  0.0021952721290290356,
  0.0022625382989645004,
  0.002440289594233036],
 'val_NSE': [0.9679062962532043,
  0.9672371745109558,
  0.9680711627006531,
  0.9652784466743469],
 'val_r_square': [0.9742463827133179,
  0.9731014370918274,
  0.9744315147399902,
  0.9744495749473572]}

In [19]:
conv_history = compile_and_fit(conv_model, conv_window)

Epoch 1/20
274/274 [==============================] - 3s 8ms/step - loss: 6.5241 - mean_squared_error: 6.5241 - NSE: -16.7991 - r_square: 0.9649 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_NSE: 0.9149 - val_r_square: 0.9240
Epoch 2/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0182 - mean_squared_error: 0.0182 - NSE: 0.9592 - r_square: 0.9651 - val_loss: 0.0074 - val_mean_squared_error: 0.0074 - val_NSE: 0.8988 - val_r_square: 0.9219
Epoch 3/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0180 - mean_squared_error: 0.0180 - NSE: 0.9581 - r_square: 0.9639 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_NSE: 0.9150 - val_r_square: 0.9249
Epoch 4/20
274/274 [==============================] - 2s 6ms/step - loss: 0.0179 - mean_squared_error: 0.0179 - NSE: 0.9592 - r_square: 0.9654 - val_loss: 0.0070 - val_mean_squared_error: 0.0070 - val_NSE: 0.9059 - val_r_square: 0.9215
Epoch 5/20
274/274 [==============================] - 

In [20]:
lstm_history = compile_and_fit(lstm_model, wide_window)

Epoch 1/20
274/274 [==============================] - 8s 16ms/step - loss: 12.9756 - mean_squared_error: 12.9756 - NSE: -34.5333 - r_square: 0.8232 - val_loss: 0.1051 - val_mean_squared_error: 0.1051 - val_NSE: -0.7004 - val_r_square: 0.9707
Epoch 2/20
274/274 [==============================] - 3s 9ms/step - loss: 0.0907 - mean_squared_error: 0.0907 - NSE: 0.7940 - r_square: 0.9707 - val_loss: 0.0152 - val_mean_squared_error: 0.0152 - val_NSE: 0.7592 - val_r_square: 0.9373
Epoch 3/20
274/274 [==============================] - 2s 8ms/step - loss: 0.0248 - mean_squared_error: 0.0248 - NSE: 0.9450 - r_square: 0.9525 - val_loss: 0.0103 - val_mean_squared_error: 0.0103 - val_NSE: 0.8351 - val_r_square: 0.9493
Epoch 4/20
274/274 [==============================] - 2s 8ms/step - loss: 0.0181 - mean_squared_error: 0.0181 - NSE: 0.9595 - r_square: 0.9644 - val_loss: 0.0098 - val_mean_squared_error: 0.0098 - val_NSE: 0.8416 - val_r_square: 0.9588
Epoch 5/20
274/274 [==============================

## Predictions

In [21]:
test_df

,Waterlevel_Sto_Nino
13140,11.89
13141,11.87
13142,11.85
13143,11.84
13144,11.84
...,...
17515,12.44
17516,12.44
17517,12.44
17518,12.44


In [24]:
test_vals = test_df.values
y_true = test_vals[1:]
y_true = y_true.reshape(-1)
y_true

array([11.87, 11.85, 11.84, ..., 12.44, 12.44, 12.44])

LSTM

In [26]:
x_test = []
y_pred = []
batch_num = 1

for x, _ in wide_window.test:
    # We need to reshape to take into account previous values
    x_numpy = x.numpy()
    x_numpy = x_numpy.reshape(len(x), x.shape[1])
    
    predictions = lstm_model.predict(x_numpy, verbose=0)
    predictions = predictions.reshape(len(predictions), predictions.shape[1])
    
    # print(x_numpy.shape)
    # print(x_numpy)
    
    # print(predictions.shape)
    
    # For the first six hours, the predictions are based on all values before them
    # For all following hours, the predictions are based on the previous 6 hours only
    if batch_num == 1:
        for i in range(len(predictions[0])):
            x_test.append(x_numpy[0][:i+1])
            y_pred.append(predictions[0][i])

        for i in range(1, len(predictions)):
            x_test.append(x_numpy[i])
            y_pred.append(predictions[i][-1])
            
        batch_num += 1
            
    else:
        for i in range(len(predictions)):
            x_test.append(x_numpy[i])
            y_pred.append(predictions[i][-1])
    
y_pred = np.array(y_pred)
y_pred

array([11.889505, 11.862694, 11.815129, ..., 12.401816, 12.401816,
       12.401816], dtype=float32)

In [27]:
len(y_pred)

4379

In [28]:
x_test[:10]

[array([11.89], dtype=float32),
 array([11.89, 11.87], dtype=float32),
 array([11.89, 11.87, 11.85], dtype=float32),
 array([11.89, 11.87, 11.85, 11.84], dtype=float32),
 array([11.89, 11.87, 11.85, 11.84, 11.84], dtype=float32),
 array([11.89, 11.87, 11.85, 11.84, 11.84, 12.26], dtype=float32),
 array([11.87, 11.85, 11.84, 11.84, 12.26, 12.63], dtype=float32),
 array([11.85, 11.84, 11.84, 12.26, 12.63, 12.81], dtype=float32),
 array([11.84, 11.84, 12.26, 12.63, 12.81, 12.8 ], dtype=float32),
 array([11.84, 12.26, 12.63, 12.81, 12.8 , 12.68], dtype=float32)]

In [30]:
SS_res = np.sum(np.square(y_true - y_pred)) 
SS_tot = np.sum(np.square(y_true - np.mean(y_true))) 

nse =  1 - SS_res/SS_tot
nse

0.9760072225691707

In [32]:
mse = np.mean(np.square(y_true-y_pred))
mse

0.007447904332009612

In [34]:
np.savetxt('univariate_lstm_predictions.csv', y_pred, delimiter=',', fmt='%.4f')

## Trials

first case: one previous water level value corresponds to one future water level value, regardless of older values

In [15]:
test_vals = test_df.values
test_vals

array([[11.89],
       [11.87],
       [11.85],
       ...,
       [12.44],
       [12.44],
       [12.44]])

In [45]:
# test_vals.shape

(4380, 1)

In [47]:
# y_pred = lstm_model.predict(test_vals)
# y_pred

137/137 [==============================] - 0s 2ms/step


array([[[11.909232]],

       [[11.889461]],

       [[11.869663]],

       ...,

       [[12.442596]],

       [[12.442596]],

       [[12.442596]]], dtype=float32)

In [51]:
# y_pred.shape

(4380, 1, 1)

### or
here, predictions take into account previous values

In [16]:
x_test_2 = test_vals.reshape(-1,6)
x_test_2

array([[11.89, 11.87, 11.85, 11.84, 11.84, 12.26],
       [12.63, 12.81, 12.8 , 12.68, 12.51, 12.36],
       [12.24, 12.18, 12.16, 12.16, 12.15, 12.13],
       ...,
       [12.45, 12.45, 12.45, 12.45, 12.45, 12.45],
       [12.44, 12.44, 12.44, 12.44, 12.44, 12.44],
       [12.44, 12.44, 12.44, 12.44, 12.44, 12.44]])

In [61]:
x_test_2.shape

(730, 6)

In [159]:
# lstm_model.predict(x_test_2)

23/23 [==============================] - 0s 4ms/step


array([[[11.909232 ],
        [11.829861 ],
        [11.80686  ],
        [11.797185 ],
        [11.802484 ],
        [12.232205 ]],

       [[12.622176 ],
        [12.802    ],
        [12.762565 ],
        [12.628821 ],
        [12.461933 ],
        [12.313349 ]],

       [[12.250967 ],
        [12.148987 ],
        [12.12417  ],
        [12.116329 ],
        [12.108276 ],
        [12.084029 ]],

       ...,

       [[12.452106 ],
        [12.428354 ],
        [12.4167595],
        [12.404326 ],
        [12.404818 ],
        [12.398731 ]],

       [[12.442596 ],
        [12.418106 ],
        [12.406666 ],
        [12.394409 ],
        [12.394964 ],
        [12.388916 ]],

       [[12.442596 ],
        [12.418106 ],
        [12.406666 ],
        [12.394409 ],
        [12.394964 ],
        [12.388916 ]]], dtype=float32)

In [187]:
# lstm_model.predict(
#     np.array([[11, 
#                11.87, 
#                11.85, 
#                11.84, 
#                11.84, 
#                12.26], 
#               [12.63, 
#                12.95, 
#                12.9, 
#                12.9, 
#                12.9, 
#                12.9]]
#     )
# )

1/1 [==============================] - 0s 25ms/step


array([[[11.004185],
        [11.860388],
        [11.778536],
        [11.80954 ],
        [11.80084 ],
        [12.233603]],

       [[12.622176],
        [12.949084],
        [12.856172],
        [12.857129],
        [12.845681],
        [12.840786]]], dtype=float32)

### or

In [175]:
for x, y in wide_window.test.take(1):
    print(tf.reshape(x[31], (-1)))
    print(x[31])
    print(lstm_model.predict(x[31]))
    print(lstm_model.predict(tf.reshape(x[31], (-1))))
    print(y[31])
    break

tf.Tensor([13.01 13.01 13.01 13.01 13.01 13.01], shape=(6,), dtype=float32)
tf.Tensor(
[[13.01]
 [13.01]
 [13.01]
 [13.01]
 [13.01]
 [13.01]], shape=(6, 1), dtype=float32)
1/1 [==============================] - 0s 21ms/step
[[[12.974106]]

 [[12.974106]]

 [[12.974106]]

 [[12.974106]]

 [[12.974106]]

 [[12.974106]]]


ValueError: in user code:

    File "C:\Users\neilb\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\neilb\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\neilb\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\neilb\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2357, in predict_step
        return self(x, training=False)
    File "C:\Users\neilb\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'sequential_2' (type Sequential).
    
    Cannot iterate over a shape with unknown rank.
